#  Informations session Spark

In [1]:
# Info session

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1682488821479_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

YARN (Yet Another Resource Negotiator) est une technologie open-source développée par Apache Hadoop qui permet de gérer les ressources et la planification des tâches dans un environnement de calcul distribué.<br><br>
Le calcul distribué est une méthode de traitement de données qui implique la répartition des tâches sur un ensemble de nœuds de calcul interconnectés, afin d'effectuer des calculs simultanément et de manière parallèle.

# Importation des packages

In [2]:
#Packages de manipulation de données
import pandas as pd
import numpy as np
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import StandardScaler, PCA
from pyspark.sql.types import ArrayType, IntegerType, FloatType, StringType
from pyspark.sql.functions import udf
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
import matplotlib.pyplot as plt
#Packages de manipulation d'images
import io
import os
from PIL import Image
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
#Packages pour l'apprentissage automatique
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split
import boto3
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import StandardScaler,PCA
from pyspark.sql.types import ArrayType, IntegerType, FloatType, StringType



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# spark.conf.set("spark.sql.parquet.writeLegacyFormat", 'false')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Définition des chemins d'accès

In [3]:
PATH = 's3a://tomato-ketchoup-p8-data'
PATH_Data = PATH+'/Test/*'
PATH_Result = PATH+'/Results/'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3a://tomato-ketchoup-p8-data
PATH_Data:   s3a://tomato-ketchoup-p8-data/Test/*
PATH_Result: s3a://tomato-ketchoup-p8-data/Results/

# Chargement de fichiers d'images JPEG

Ce code utilise Spark pour charger des fichiers d'images au format JPEG à partir d'un répertoire spécifié dans PATH_Data. Les images sont lues en utilisant l'API binaryFile de Spark, qui permet de charger des fichiers binaires tels que des images, des fichiers audio ou vidéo, etc.<br><br>
Les options de configuration spécifiées incluent le filtrage des fichiers pour ne charger que les fichiers avec l'extension ".jpg" et la recherche récursive de fichiers à partir du chemin spécifié. Le résultat est stocké dans la variable images.

In [4]:
images = spark.read.format('binaryFile') \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
images.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3a://tomato-ketc...|2023-04-24 12:09:07|  7353|[FF D8 FF E0 00 1...|
|s3a://tomato-ketc...|2023-04-24 12:09:07|  7350|[FF D8 FF E0 00 1...|
|s3a://tomato-ketc...|2023-04-24 12:09:07|  7349|[FF D8 FF E0 00 1...|
|s3a://tomato-ketc...|2023-04-24 12:09:06|  7348|[FF D8 FF E0 00 1...|
|s3a://tomato-ketc...|2023-04-24 12:09:08|  7328|[FF D8 FF E0 00 1...|
|s3a://tomato-ketc...|2023-04-24 12:09:08|  7301|[FF D8 FF E0 00 1...|
|s3a://tomato-ketc...|2023-04-24 12:09:08|  7278|[FF D8 FF E0 00 1...|
|s3a://tomato-ketc...|2023-04-24 12:09:08|  7275|[FF D8 FF E0 00 1...|
|s3a://tomato-ketc...|2023-04-24 12:09:08|  7266|[FF D8 FF E0 00 1...|
|s3a://tomato-ketc...|2023-04-24 12:09:08|  7266|[FF D8 FF E0 00 1...|
|s3a://tomato-ketc...|2023-04-24 12:09:08|  7265|[FF D8 FF E0 00 1...|
|s3a:/

Le DataFrame comporte plusieurs colonnes :

- path : le chemin complet du fichier<br><br>
- modificationTime : l'heure de la dernière modification du fichier<br><br>
- length : la taille du fichier en octets<br><br>
- content : le contenu du fichier (en octets) sous forme de tableau d'octets

Chaque ligne du DataFrame correspond à un fichier présent dans l'espace de stockage en nuage AWS S3 spécifié dans path.

## Ajout catégorie

In [7]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Ce code ajoute une nouvelle colonne appelée "label" au DataFrame "images" créé à partir des fichiers JPG stockés dans un emplacement AWS S3.

Le DataFrame "images" a une colonne "path" qui contient le chemin complet des fichiers JPG. Cette colonne "path" est divisée en parties en utilisant le caractère de séparation "/" avec la fonction "split". Les parties sont ensuite stockées dans un tableau.

La fonction "element_at" est utilisée pour sélectionner l'élément du tableau correspondant à l'avant-dernière position.

Dans ce cas, l'avant-dernière partie du chemin est extraite et stockée dans une nouvelle colonne appelée "label".

In [8]:
images.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+----------+
|                path|   modificationTime|length|             content|     label|
+--------------------+-------------------+------+--------------------+----------+
|s3a://tomato-ketc...|2023-04-24 12:09:07|  7353|[FF D8 FF E0 00 1...|Watermelon|
|s3a://tomato-ketc...|2023-04-24 12:09:07|  7350|[FF D8 FF E0 00 1...|Watermelon|
+--------------------+-------------------+------+--------------------+----------+
only showing top 2 rows

In [11]:
print(images.printSchema())
print(images.select('path','label').show(5,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+-----------------------------------------------------------+----------+
|path                                                       |label     |
+-----------------------------------------------------------+----------+
|s3a://tomato-ketchoup-p8-data/Test/Watermelon/r_106_100.jpg|Watermelon|
|s3a://tomato-ketchoup-p8-data/Test/Watermelon/r_109_100.jpg|Watermelon|
|s3a://tomato-ketchoup-p8-data/Test/Watermelon/r_108_100.jpg|Watermelon|
|s3a://tomato-ketchoup-p8-data/Test/Watermelon/r_107_100.jpg|Watermelon|
|s3a://tomato-ketchoup-p8-data/Test/Watermelon/r_95_100.jpg |Watermelon|
+-----------------------------------------------------------+----------+
only showing top 5 rows

None

# Configuration modèle


## Initialisation d'un modèle de réseau de neurones 

Le code initialise un modèle de réseau de neurones convolutif appelé MobileNetV2, qui a été pré-entraîné sur le jeu de données ImageNet.

Le modèle prend en entrée des images de taille (224, 224, 3)

MobileNetV2 est une architecture de réseau de neurones convolutif conçue pour une utilisation sur des appareils mobiles.

Elle est légère et peut donc être exécutée sur des appareils ayant des ressources limitées, tout en obtenant de bonnes performances de reconnaissance d'images.

In [9]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14536120/14536120 [==============================] - 1s 0us/step

Le code crée un nouveau modèle en utilisant la classe Model de Keras. Le nouveau modèle est basé sur le modèle pré-entraîné MobileNetV2 qui a été chargé avec les poids pré-entraînés sur l'ensemble de données ImageNet. Les "poids" dans ce contexte font référence aux valeurs numériques qui définissent les relations entre les neurones du réseau. 

Le nouveau modèle est construit en spécifiant que son entrée est la même que celle du modèle MobileNetV2 et que sa sortie est la couche avant la dernière couche de classification du modèle original.

En d'autres termes, la dernière couche de classification est supprimée et la sortie du nouveau modèle est la sortie de la couche précédente. 

Ainsi, le nouveau modèle peut être utilisé pour extraire des caractéristiques à partir des images, qui peuvent être utilisées comme entrée pour d'autres tâches de traitement d'images telles que la classification ou la segmentation

In [10]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Avec la méthode sc.broadcast() de Spark, les poids du modèle sont envoyés une seule fois depuis le nœud maître (master) à chaque nœud du cluster. Une fois que les poids ont été envoyés à tous les nœuds, ils peuvent être utilisés de manière locale sur chaque nœud pour effectuer des calculs d'entraînement en parallèle sur différentes parties du jeu de données.

Cette technique est couramment utilisée dans l'apprentissage distribué pour distribuer les données et les paramètres du modèle sur plusieurs nœuds pour des calculs plus rapides et plus efficaces. De cette manière, chaque nœud dispose d'une copie locale des poids et peut effectuer des calculs sur des parties différentes du jeu de données, puis renvoyer les résultats au nœud maître pour agréger les mises à jour du modèle.

In [11]:
brodcast_weights = sc.broadcast(new_model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Récapitulatif modèle

In [12]:
new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

# Définitions fonctions

Création d'un modèle MobileNetV2 avec M
- les poids pré-entraînés <br>

- les couches de classification supprimées <br>

- les poids transmis via Broadcast en utilisant PySpark

In [13]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Fonction de prétraitement d'image pour la prédiction avec un modèle de Deep Learning

Cette fonction preprocess prend en entrée le contenu brut d'une image et effectue les opérations de prétraitement nécessaires.

Elle utilise la bibliothèque PIL (Python Imaging Library) pour ouvrir l'image en mémoire, la redimensionner à une taille de (224, 224) et la convertir en un tableau Numpy.

Ensuite, elle utilise la fonction preprocess_input de Keras pour normaliser les valeurs des pixels de l'image selon les critères utilisés lors de l'entraînement du modèle.

In [14]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Fonction de featurization pour les pd.Series d'images brutes

La fonction featurize_series est utilisée pour effectuer la featurisation sur chaque batch d'images. 

Elle redimensionne chaque image à la taille d'entrée attendue par le modèle (224x224), puis applique une fonction de prétraitement qui normalise les pixels de l'image en fonction des valeurs moyennes et d'écart-type de l'ensemble de données ImageNet sur lequel le modèle a été pré-entraîné. 

In [20]:
def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input_ = np.stack(content_series.map(preprocess))
    feats = model.predict(input_)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in feats]
    return pd.Series(output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Fonction UDF de featurisation d'images avec le modèle MobileNetV2 dans PySpark"

Cette fonction est utilisée pour featuriser un grand nombre d'images à l'aide d'un modèle de réseau de neurones.

Elle featurise chaque image à l'aide du modèle défini dans la fonction model_fn, et renvoie une pandas Series de vecteurs de features (une représentation numérique des images obtenue à partir des sorties du modèle).

En utilisant la fonction featurize_udf, on peut appliquer cette opération de featurisation sur de grandes quantités d'images en parallèle à l'aide d'Apache Spark.

In [21]:
@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    Retourne une Spark DataFrame colonne de type ArrayType(FloatType) contenant les features obtenues grâce au ResNet50.
    'content_series_iter' permet d'itérer sur les batchs de données contenant les images d'entrées.
    
    '''
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

la valeur de spark.sql.execution.arrow.maxRecordsPerBatch est définie sur 34. 

Cela signifie que la fonctionnalité Arrow récupérera jusqu'à 34 lignes à la fois lors de la transmission des données entre Spark et Python. 

Cette valeur a été choisie pour des raisons de performance et de contraintes de mémoire spécifiques à ce cas d'utilisation particulier.

In [22]:
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "34")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Création d'un DataFrame de features à partir d'un DataFrame d'images

In [23]:
features_df = images.repartition(24).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
features_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+--------------------+
|                path|     label|            features|
+--------------------+----------+--------------------+
|s3a://tomato-ketc...|Watermelon|[0.0, 0.9346366, ...|
|s3a://tomato-ketc...|Watermelon|[0.029508123, 0.0...|
|s3a://tomato-ketc...|Watermelon|[0.8537648, 0.450...|
|s3a://tomato-ketc...|Watermelon|[0.77535754, 0.24...|
|s3a://tomato-ketc...|Watermelon|[0.32722184, 0.06...|
|s3a://tomato-ketc...|Watermelon|[0.007843309, 0.0...|
|s3a://tomato-ketc...|Watermelon|[0.046800975, 0.0...|
|s3a://tomato-ketc...|     Lemon|[0.011176158, 0.0...|
|s3a://tomato-ketc...|     Lemon|[0.09697362, 0.0,...|
|s3a://tomato-ketc...|     Lemon|[0.031395115, 0.0...|
|s3a://tomato-ketc...|     Lemon|[0.046849318, 0.0...|
|s3a://tomato-ketc...|     Lemon|[0.2037401, 0.152...|
|s3a://tomato-ketc...|     Lemon|[0.36607733, 0.00...|
|s3a://tomato-ketc...|     Lemon|[0.36865997, 9.10...|
|s3a://tomato-ketc...|Watermelon|[0.12482068, 0.02...|
|s3a://tom

# Réduction de dimension

Le but de cette opération est de convertir la colonne "features" de type ArrayType(FloatType) en une colonne de type Vector pour pouvoir appliquer la méthode PCA (Principal Component Analysis) qui nécessite des vecteurs d'entrée.

La fonction udf utilisée ici permet de transformer chaque array de features en un vecteur dense de type Vectors.dense() qui peut être traité par la méthode PCA.

In [25]:
# De Array à Vectors pour appliquer la PCA
array_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

vectorized_df est le résultat de la transformation de features_df en remplaçant la colonne features par une nouvelle colonne appelée features_vectors.

La nouvelle colonne est obtenue à partir de la colonne features en appliquant la fonction array_to_vector_udf() sur chaque élément de la colonne features à l'aide de la méthode withColumn().

La fonction array_to_vector_udf() est une fonction utilisateur définie (UDF) qui prend une liste d'éléments (ici, les features de chaque image) et la transforme en un objet Vector de PySpark.

Ainsi, chaque élément de la colonne features est converti en un objet Vector, permettant ainsi l'application de la PCA sur les features de chaque image.

In [26]:
vectorized_df = features_df.withColumn('features_vectors', array_to_vector_udf('features'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
# reduce with PCA - set k Max to determine the adequate nb of principal components

pca = PCA(k=20, inputCol='features_vectors', outputCol='pca_vectors')
model = pca.fit(vectorized_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
pca_df = model.transform(vectorized_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
pca_df.show(5, True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+--------------------+--------------------+--------------------+
|                path|     label|            features|    features_vectors|         pca_vectors|
+--------------------+----------+--------------------+--------------------+--------------------+
|s3a://tomato-ketc...|Watermelon|[0.0, 0.9346366, ...|[0.0,0.9346365928...|[-15.459348989296...|
|s3a://tomato-ketc...|Watermelon|[0.029508123, 0.0...|[0.02950812317430...|[-16.446105694734...|
|s3a://tomato-ketc...|Watermelon|[0.8537648, 0.450...|[0.85376477241516...|[-15.311455874879...|
|s3a://tomato-ketc...|Watermelon|[0.77535754, 0.24...|[0.77535754442214...|[-15.140514399676...|
|s3a://tomato-ketc...|Watermelon|[0.32722184, 0.06...|[0.32722184062004...|[-17.860688546459...|
+--------------------+----------+--------------------+--------------------+--------------------+
only showing top 5 rows

# Enregistrement des résultats

In [57]:
print(PATH_Result)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3a://tomato-ketchoup-p8-data/Results/

In [58]:
pca_df.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
df_spark = spark.read.parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
df_spark.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+--------------------+--------------------+--------------------+
|                path|     label|            features|    features_vectors|         pca_vectors|
+--------------------+----------+--------------------+--------------------+--------------------+
|s3a://tomato-ketc...|Watermelon|[0.60736495, 0.06...|[0.60736495256423...|[-15.713608145722...|
|s3a://tomato-ketc...|Watermelon|[0.018489283, 0.0...|[0.01848928257822...|[-13.521456319804...|
|s3a://tomato-ketc...|Watermelon|[0.5296135, 0.097...|[0.52961349487304...|[-17.366385363900...|
|s3a://tomato-ketc...|Watermelon|[0.015380343, 0.3...|[0.01538034342229...|[-12.684613674608...|
|s3a://tomato-ketc...|Watermelon|[0.13633335, 0.20...|[0.13633334636688...|[-15.540649931117...|
|s3a://tomato-ketc...|Watermelon|[0.0, 0.2200655, ...|[0.0,0.2200655043...|[-15.209976527943...|
|s3a://tomato-ketc...|Watermelon|[0.0, 0.274615, 0...|[0.0,0.2746149897...|[-14.122554504376...|
|s3a://tomato-ketc...|Watermel

In [67]:
print("Number of rows: ", df_spark.count())
print("Number of columns: ", len(df_spark.columns))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of rows:  321
Number of columns:  5

In [75]:
len(df_spark.select(col("features")).first()[0])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1280